# Imports

In [ ]:
# Standard library imports
import sys
import os

# Add path
sys.path.append(os.path.join('D:\\','OneDrive','Dokumente','Python','BMCSim'))

# Third party imports
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import HTML

# Local application imports
from BMCtool import BMCTool

# settings

In [ ]:
# simulation settings
n_timesteps = 100  # number of timesteps for each RF pulse
track = True  # track the magnetization trajectory during pulses/delays ?!
par_calc = True  # calculate all offsets in parallel instead of subsequently

# experimental settings
B0 = 7  # B0 in T
B1 = 1.25e-6*B0  # B1 in T (for WASABI use 1.25e-6*B0)
n_p = 1  # number of saturation pulses
tp = 0.015/B0  # pulse duration in s (for WASABI use 0.015/B0)
td = 0  # delay between pulses in s
shape = 'CW'
trec = 0  # recovery/delay time between offsets in s
offset = 2  # max offset in ppm
n_offsets = 101  # number of offsets

# sample settings
T1 = 2            #T1 in s
T2 = 0.05            #T2 in s

# CEST settings
n_pools = 2

#                bulk       1st       2nd       3rd       4th       5th       6th
T1n = np.array([   T1,       T1,       T1,       T1,       T1,       T1,       T1])
T2n = np.array([   T2,     0.15,     9e-6,     0.15,    0.015,     9e-6,    0.015])
fn  = np.array([    1,    0.002,     0.14,    0.005,    0.005,     0.14,     0.01])
dwn = np.array([ -0.0,      1.3,     -2.6,     -3.3,     -3.7,     -2.6,     -1.0])
kn  = np.array([    0,     2000,       40,     2000,     2000,       40,      500])

# calculate offsets
offsets = np.linspace(-offset, offset, n_offsets)

# simulation

In [ ]:
# create Sim object
Sim = BMCTool(b0=B0,
              n_pools=n_pools,
              t1_values=T1n,
              t2_values=T2n,
              poolsizes=fn,
              resonances=dwn,
              exchange_rates=kn,
              track=track)

if par_calc:
    # set initial magnetization
    Sim.set_M(offsets.shape[0])
 
    # simulate recovery
    Sim.solve(offsets, 0, trec, n_timesteps, shape=shape)

    # simulate saturation pulse train
    for n in range(n_p):
        # delay between pulses (not for the first pulse)
        if n != 0 and td != 0:
            Sim.solve(offsets, b1=0, pulse_dur=td, steps=int(td/tp*n_timesteps))
        # saturation
        Sim.solve(offsets, b1=B1, pulse_dur=tp, steps=int(n_timesteps), shape=shape)
        
    M = Sim.M_
else:
    # create array for magnetization values
    M = np.zeros([len(offsets), 3*n_pools, 1])
    
    # set initial magnetization
    Sim.set_M(1)

    for i in range(offsets.shape[0]):
        # simulate recovery
        if trec > 0:
            Sim.solve(offsets[i], 0, trec, n_timesteps, shape='PAUSE')

        # simulate saturation pulse train
        for n in range(n_p):
            if n != 0 and td != 0:
                Sim.solve(offsets[i], b1=0, pulse_dur=td, steps=int(td/tp*n_timesteps))
            Sim.solve(offsets[i], b1=B1, pulse_dur=tp, steps=int(n_timesteps), shape=shape)
        
        # write magnetization in array
        M[i,] = Sim.M_

# Plot spektrum (Signal vs. offsets) and Asymmetry

In [ ]:
%matplotlib inline
fig, ax = plt.subplots(figsize=(12,9))

ax.plot(offsets, M[:,2,0], marker='o', linestyle='--', linewidth=2, color='black')
ax.set_xlabel('frequency offset [ppm]', fontsize=20)
ax.set_ylabel('normalized signal', fontsize=20)
ax.set_ylim([-0.1,1])
ax.invert_xaxis()
ax.grid()

# substract left from right side
asym = M[::-1,2,0] - M[:,2,0]
# get indices of possitive offsets
idx = np.where(np.logical_and(offsets >= 0, offsets <= np.amax(offsets)))
# plot asymmetry data
ax.plot(offsets[idx],asym[idx],color='r',linestyle='--')

# Plot (on-resonant) time curve (only available if track=True)

In [ ]:
%matplotlib qt

# find index closest to CEST resonance
#idx = int(np.where(offsets == offsets[np.abs(offsets - dwn[1]).argmin()])[0])
idx = 0

fig, ax = plt.subplots(figsize=(12,9))
l1 = ax.plot(Sim.t, Sim.Mt[:,idx,2,0], marker='o', markersize=8, linestyle='-', linewidth=2)
ax.set_xlabel('time [s]', fontsize=20)
ax.set_ylabel('normalized signal', fontsize=20)
ax.tick_params(labelsize=18)

# Plot magnetization trajectory

In [ ]:
%matplotlib inline

fig = plt.figure(figsize=(15,10))
ax = fig.add_subplot(111, projection='3d')
ax.plot3D(Sim.Mt[:,idx,0,0], Sim.Mt[:,idx,1,0], Sim.Mt[:,idx,2,0], 'red')